In [1]:
# 데이터는 CIFAR10 10가지 분류....
# 모델을만들어서
# 분류하고자한는 이미지가 폴더에 있을때. 해당 폴더로 자동으로 분류(이동)

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import ResNet50

In [24]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
# 정규화
x_train = x_train / 255.0
x_test = x_test / 255.0

In [4]:
x_train.shape[1:]

(32, 32, 3)

In [5]:
y_train

array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [26]:
from tensorflow.python.framework import tensor
base_model = ResNet50(weights='imagenet', include_top=False, input_shape = x_train.shape[1:])
# 가중치 동결하지 않는 경우 기존 가중치와 함께 학습이 되어서
# 훈련데이터가 충분하지 않으면 기존 가중치의 영향에 묻혀서 제대로 반영이 안될수 있음

# 기존 모델의 가중치를 동결 해제
for layer in base_model.layers:
  layer.trainable = True

# 마지막에 우리 분류기 넣는다.
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256,activation='relu'),
    layers.Dense(10,activation='softmax')
])
model.compile(optimizer='adam', loss = tf.keras.losses.sparse_categorical_crossentropy,metrics=['accuracy'])
model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
1563/1563 [==============================] - 121s 52ms/step - loss: 1.3722 - accuracy: 0.5368 - val_loss: 1.3279 - val_accuracy: 0.5248
Epoch 2/10
1563/1563 [==============================] - 80s 51ms/step - loss: 1.1805 - accuracy: 0.5948 - val_loss: 2.3247 - val_accuracy: 0.4771
Epoch 3/10
1563/1563 [==============================] - 76s 49ms/step - loss: 1.0470 - accuracy: 0.6450 - val_loss: 1.1283 - val_accuracy: 0.6194
Epoch 4/10
1563/1563 [==============================] - 91s 58ms/step - loss: 0.8935 - accuracy: 0.6996 - val_loss: 1.2287 - val_accuracy: 0.5814
Epoch 5/10
1563/1563 [==============================] - 77s 49ms/step - loss: 0.9350 - accuracy: 0.6799 - val_loss: 2.3567 - val_accuracy: 0.3897
Epoch 6/10
1563/1563 [==============================] - 81s 52ms/step - loss: 0.8166 - accuracy: 0.7213 - val_loss: 1.1047 - val_accuracy: 0.6511
Epoch 7/10
1563/1563 [==============================] - 80s 51ms/step - loss: 0.6745 - accuracy: 0.7720 - val_loss: 0.9539 

In [ ]:
model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

In [ ]:
model.save('resnet50_cifar10.h5')
model = keras.models.load_model('resnet50_cifar10.h5')

In [ ]:
!unzip '/content/test_images.zip' -d 'test_imgs'

In [9]:
import numpy as np
from PIL import Image
from glob import glob

In [10]:
img_files = glob('./test_imgs/*.jpg')
img_files[:3]

['./test_imgs/12.jpg', './test_imgs/04.jpg', './test_imgs/02.jpg']

In [19]:
img = Image.open(img_files[0])
x_tests = []
x = np.array(img.resize([32,32])) / 255.0
x_tests.append(x)
x_tests = np.array(x_tests)
np.argmax(model.predict(x_tests))

1/1 [==============================] - 0s 34ms/step


3

In [27]:
classs_names = ['airplane',
'automobile'							,
'bird'										,
'cat'										,
'deer'									,
'dog'										,
'frog'									,
'horse'									,
'ship'									,
'truck']
classs_names

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [38]:
import os
tests = []
origin_images = []
for file in img_files:
  img = Image.open(file)
  origin_images.append(img)
  x = np.array(img.resize([32,32])) / 255.0
  tests.append(x)
tests = np.array(tests)
print(tests.shape)
preds = model.predict(tests)
print(len(preds))
pred_index =  [np.argmax(pred) for pred in preds]
print(pred_index)
for i, idx in enumerate(pred_index):  # i는 예측한 이미지의 순서 idx는 해당 클래스
  if not os.path.isdir(classs_names[idx]):
    os.mkdir(classs_names[idx])

  os.chdir(classs_names[idx])

  origin_images[i].save( img_files[i].split('/')[-1] )
  os.chdir('../')

(12, 32, 32, 3)
1/1 [==============================] - 0s 26ms/step
12
[7, 6, 2, 1, 4, 6, 8, 2, 8, 8, 8, 6]


In [37]:
len(origin_images)

12